In [1]:
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName("transfrom").getOrCreate()
spark

25/04/28 13:28:22 WARN Utils: Your hostname, aayushgyawali resolves to a loopback address: 127.0.1.1; using 10.10.42.111 instead (on interface enp2s0)
25/04/28 13:28:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/28 13:28:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.option("multiline",True).json("provider.json")
df.show()



df.printSchema()

+-----------------+--------------------+
|provider_group_id|     provider_groups|
+-----------------+--------------------+
|         10001001|[{[1235233008, 13...|
|         10002001|[{[1609829761, 18...|
|         10003001|[{[1720350119], {...|
|         10004001|[{[1417152349, 16...|
|         10006001|[{[1043293582, 19...|
|         10007001|[{[1275520645], {...|
|         10008001|[{[1114026911], {...|
|         10009001|[{[1003084302, 10...|
|         10010001|[{[1710182167], {...|
|         10011001|[{[1326039934, 14...|
|         10012001|[{[1023093721, 12...|
|         10013001|[{[1003127622, 10...|
|         10014001|[{[1871582346], {...|
|         10015001|[{[1194743344, 12...|
|         10016001|[{[1003038530, 10...|
|         10017001|[{[1063644110, 10...|
|         10018001|[{[1003016411, 10...|
|         10019001|[{[1083710446, 11...|
|         10020001|[{[1740552942], {...|
|         10021001|[{[1477653954], {...|
+-----------------+--------------------+
only showing top

In [4]:

from pyspark.sql.functions import col, explode

provider_df = df.withColumn("row", explode("provider_groups"))

provider_flat = provider_df.select(
    col("provider_group_id"),
    col("row.npi").alias("npi"),
    col("row.tin.type").alias("tin_type"),
    col("row.tin.value").alias("tin_value")
)

provider_flat.show(truncate=False)

+-----------------+------------------------------------------------+--------+----------+
|provider_group_id|npi                                             |tin_type|tin_value |
+-----------------+------------------------------------------------+--------+----------+
|10001001         |[1235233008, 1316041189, 1780788554, 1891068409]|ein     |04-3267217|
|10001001         |[1366459570]                                    |ein     |11-1562701|
|10001001         |[1417915653]                                    |ein     |11-3358535|
|10001001         |[1417915653]                                    |ein     |13-3888838|
|10002001         |[1609829761, 1821482241]                        |ein     |00-0004110|
|10002001         |[1760986277]                                    |ein     |00-6980743|
|10002001         |[1215075882]                                    |ein     |01-0550744|
|10002001         |[1013917665]                                    |ein     |01-0555304|
|10002001         |[1

In [5]:
from pyspark.sql.functions import regexp_replace,col,expr,monotonically_increasing_id,hash,lit

hypenrm_df=provider_flat.withColumn("tin_value",regexp_replace(col("tin_value"),"-",""))
hypenrm_df.show()

+-----------------+--------------------+--------+---------+
|provider_group_id|                 npi|tin_type|tin_value|
+-----------------+--------------------+--------+---------+
|         10001001|[1235233008, 1316...|     ein|043267217|
|         10001001|        [1366459570]|     ein|111562701|
|         10001001|        [1417915653]|     ein|113358535|
|         10001001|        [1417915653]|     ein|133888838|
|         10002001|[1609829761, 1821...|     ein|000004110|
|         10002001|        [1760986277]|     ein|006980743|
|         10002001|        [1215075882]|     ein|010550744|
|         10002001|        [1013917665]|     ein|010555304|
|         10002001|[1679780811, 1700...|     ein|010555483|
|         10002001|        [1376647511]|     ein|010567880|
|         10002001|        [1740287010]|     ein|010569673|
|         10002001|        [1265538219]|     ein|010571165|
|         10002001|        [1689625253]|     ein|010577025|
|         10002001|        [1184901563]|

In [6]:
# hypenrm_df.write.json('pr.json')
# hypenrm_df.toPandas().to_json('output.json', orient='records')
hypenrm_df.printSchema()

root
 |-- provider_group_id: long (nullable = true)
 |-- npi: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- tin_type: string (nullable = true)
 |-- tin_value: string (nullable = true)

